## Inpainting with Variable Dataset Size

Recall that the ImageWang dataset consists of:

1. A `/val` folder with 10 classes.
2. A `/train` folder with 20 classes. 
  - There are ~125 images in each class that exists in `/val`. There are 
  - There are ~1,300 images in each class that does not exist in `/val`
3. An `/unsup` folder with 7,750 unlabelled images.

The question we would like to answer with this notebook is:

> What is the effect of dataset size during pretext training on downstream task performance?

To answer this question we will consider four different datasets, each built from ImageWang.

They are:

1. All data in `/train`, `/unsup` and `/val`
2. All data in `/train`, `/unsup`
3. All data in `/train`
4. Only Data in `/train` that has a corresponding class in `/val`

In [1]:
import json

import numpy as np

from fastai2.basics import DataBlock, RandomSplitter

from fastai2.vision.all import ImageBlock, PILMask, get_image_files, PILImage, imagenet_stats
from fastai2.vision.core import get_annotations, Image, TensorBBox, TensorPoint, TensorImage
from fastai2.vision.augment import aug_transforms, RandomResizedCrop, RandTransform
from fastai2.vision.learner import unet_learner, unet_config

from fastai2.data.transforms import Normalize
from fastai2.data.external import download_url, URLs, untar_data

from torch.nn import MSELoss
from torchvision.models import resnet34